In [1]:
!pip3.12 install albumentations
!pip3.12 install ultralytics


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:

import os
import cv2
import numpy as np
import albumentations as A
from ultralytics import YOLO

class HomomorphicFilter:
    def __init__(self, a=0.5, b=1.5):
        self.a, self.b = float(a), float(b)
    def __butterworth_filter(self, shape, params):
        P, Q = shape[0]//2, shape[1]//2
        U, V = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
        D = (U-P)**2 + (V-Q)**2
        H = 1.0 / (1.0 + (D/(params[0]**2))**params[1])
        return 1.0 - H
    def __gaussian_filter(self, shape, params):
        P, Q = shape[0]//2, shape[1]//2
        U, V = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
        D = (U-P)**2 + (V-Q)**2
        H = np.exp(-D/(2*(params[0]**2)))
        return 1.0 - H
    def __apply_filter(self, I_fft, H):
        Hs = np.fft.fftshift(H)
        return (self.a + self.b*Hs) * I_fft
    def filter(self, I, filter_params, mode='butterworth', H_ext=None):
        if I.ndim != 2:
            raise ValueError("Input must be single-channel")
        I_log = np.log1p(I.astype(float))
        I_fft = np.fft.fft2(I_log)
        if mode == 'butterworth':
            H = self.__butterworth_filter(I_fft.shape, filter_params)
        elif mode == 'gaussian':
            H = self.__gaussian_filter(I_fft.shape, filter_params)
        elif mode == 'external' and H_ext is not None:
            H = H_ext
        else:
            raise ValueError(f"Unknown filter mode {mode}")
        I_filt = np.fft.ifft2(self.__apply_filter(I_fft, H))
        I_out = np.exp(np.real(I_filt)) - 1
        return np.uint8(np.clip(I_out, 0, 255))


def relief_transform(img: np.ndarray, bias: int = 128) -> np.ndarray:
    if img.ndim != 2:
        raise ValueError("Expect grayscale")
    h, w = img.shape
    out = np.zeros((h, w), dtype=np.int16)
    out[1:-1,1:-1] = (
        img[:-2,:-2].astype(int)
        - img[2:,2:].astype(int)
        + bias
    )
    out = np.clip(out, 0, 255).astype(np.uint8)
    out[0,:], out[-1,:], out[:,0], out[:,-1] = img[0,:], img[-1,:], img[:,0], img[:,-1]
    return out


class HECRTransform(A.ImageOnlyTransform):
    def __init__(self, clip_limit=5.0, bias=128, p=1.0):
        super().__init__(p=p)
        self.clip_limit = clip_limit
        self.bias = bias
        self.homo = HomomorphicFilter(a=0.75, b=1.25)
    def apply(self, img, **kwargs):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eroded = cv2.erode(img, np.ones((3,3), np.uint8), iterations=1)
        h = self.homo.filter(eroded[:,:,0], filter_params=[30,2])
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit)
        ch1 = clahe.apply(h)
        rel = relief_transform(gray, self.bias)
        ch2 = clahe.apply(rel)
        return np.stack([gray, ch1, ch2], axis=2)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

WEIGHTS    = 'last-4.pt'
IMAGE_PATH = '13-350-ls-34-g01.png'
OUTPUT_DIR = 'outputs'
IMG_SIZE   = 1024
CONF_THR   = 0.05
TILE       = 1140
STRIDE     = int(TILE*0.8)
REGIONS    = 30
CLASS_NAMES = {
 0:"пора",1:"включение",2:"подрез",3:"прожог",4:"трещина",5:"наплыв",
 6:"эталон1",7:"эталон2",8:"эталон3",9:"пора-скрытая",10:"утяжина",
 11:"несплавление",12:"непровар корня"}

def predict_masks_and_regions():
    os.makedirs(OUTPUT_DIR,exist_ok=True)
    model = YOLO(WEIGHTS)
    img   = cv2.imread(IMAGE_PATH); assert img is not None, 'image not found'
    H,W   = img.shape[:2]; region_w = W/REGIONS
    transformer = A.Compose([HECRTransform(p=1.0)])

    region_rows, pred_lines = [], []
    overlay = img.copy()

    y0,y1 = 0,TILE
    for x0 in range(0, W-TILE+1, STRIDE):
        crop = img[y0:y1, x0:x0+TILE]
        proc = transformer(image=crop)['image']
        res  = model.predict(proc, imgsz=IMG_SIZE, conf=CONF_THR,
                             task='segment', verbose=False)[0]
        if res.masks is None:
            continue
        masks   = res.masks.data.cpu().numpy()            # (N,mh,mw)
        classes = res.boxes.cls.cpu().numpy().astype(int)
        confs   = res.boxes.conf.cpu().numpy()
        mh,mw   = masks.shape[1:]

        for m,cls,conf in zip(masks,classes,confs):
            m_big = cv2.resize(m,(TILE,TILE),interpolation=cv2.INTER_NEAREST)
            mask_full = np.zeros((H,W),np.uint8); mask_full[y0:y1,x0:x0+TILE]=m_big*255
            color = np.random.randint(0,255,(3,),dtype=np.uint8)
            overlay[mask_full>0] = overlay[mask_full>0]*0.5 + color*0.5
            cnts,_ = cv2.findContours(m_big.astype(np.uint8),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            if not cnts:
                continue
            cnt   = max(cnts,key=cv2.contourArea).reshape(-1,2)
            cnt[:,0]+=x0; cnt[:,1]+=y0       
            poly_str = ' '.join(f'{x:.2f} {y:.2f}' for x,y in cnt)
            pred_lines.append(f"{cls} {poly_str}")
            regs = set((cnt[:,0]//region_w).astype(int))
            for r in regs:
                region_rows.append({'region':int(r),
                                    'defect':CLASS_NAMES.get(int(cls),str(cls))})


    txt_path = os.path.join(OUTPUT_DIR,'predictions.txt')
    with open(txt_path,'w',encoding='utf-8') as f:
        f.write('\n'.join(pred_lines))


    df = pd.DataFrame(region_rows)

    agg = (df.groupby('region')['defect']
             .apply(lambda s: ','.join(sorted(set(s))))
             .reset_index())
    full = pd.DataFrame({'region':range(REGIONS)}).merge(agg, how='left')
    full.to_csv(os.path.join(OUTPUT_DIR,'report.csv'),index=False,encoding='utf-8')


    cv2.imwrite(os.path.join(OUTPUT_DIR,'masks_overlay.png'), overlay)

In [5]:
predict_masks_and_regions()

✅ 27 polygons → predictions.txt
✅ report.csv saved (all 30 regions)
✅ masks_overlay.png saved
